In [4]:
#unzip the data 
import zipfile
# Unzip the downloaded file
zip_ref = zipfile.ZipFile("/content/Sarcasm_Headlines_Dataset.json.zip", "r")
zip_ref.extractall()
zip_ref.close()

In [6]:
import pandas as pd
df = pd.read_json('/content/Sarcasm_Headlines_Dataset.json',lines=True)

In [7]:
df.head()


,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [8]:
df = df.drop(['article_link'], axis=1)#Droping the article_link coloum 
df.head() 

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


In [9]:
X = df['headline']

In [10]:
X

0        former versace store clerk sues over secret 'b...
1        the 'roseanne' revival catches up to our thorn...
2        mom starting to fear son's web series closest ...
3        boehner just wants wife to listen, not come up...
4        j.k. rowling wishes snape happy birthday in th...
                               ...                        
26704                 american politics in moral free-fall
26705                              america's best 20 hikes
26706                                reparations and obama
26707    israeli ban targeting boycott supporters raise...
26708                    gourmet gifts for the foodie 2014
Name: headline, Length: 26709, dtype: object

In [11]:
Y = df['is_sarcastic']

In [12]:
Y

0        0
1        0
2        1
3        1
4        0
        ..
26704    0
26705    0
26706    0
26707    0
26708    0
Name: is_sarcastic, Length: 26709, dtype: int64

In [13]:
import tensorflow  as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization  

In [14]:
# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 10 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [15]:
text_vectorizer.adapt(X)


In [16]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1") 

embedding

#

In [17]:
#functional Aip 
#model creation
tf.random.set_seed(42)
from tensorflow.keras import layers
Inputs =  layers.Input(shape=(1,),dtype='string')
x = text_vectorizer(Inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(3,activation='relu')(x)
#x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1,activation='sigmoid')(x)
Dense_model = tf.keras.Model(Inputs,outputs,name='Dens_model')

In [18]:
#model compile 
Dense_model.compile(loss="binary_crossentropy",
                    optimizer =tf.keras.optimizers.Adam(),
                    metrics=['accuracy'])

In [ ]:
from sklearn.model_selection import KFold

n_split=3

for train_index,test_index in KFold(n_split).split(X):
  x_train,x_test=X[train_index],X[test_index]
  y_train,y_test=Y[train_index],Y[test_index]

  Dense_model.fit(x_train, y_train,epochs=30)

Epoch 1/30
557/557 [==============================] - 5s 4ms/step - loss: 0.4636 - accuracy: 0.7886
Epoch 2/30
557/557 [==============================] - 3s 4ms/step - loss: 0.2639 - accuracy: 0.8930
Epoch 3/30
557/557 [==============================] - 2s 4ms/step - loss: 0.1930 - accuracy: 0.9264
Epoch 4/30
557/557 [==============================] - 2s 4ms/step - loss: 0.1504 - accuracy: 0.9468
Epoch 5/30
557/557 [==============================] - 2s 4ms/step - loss: 0.1208 - accuracy: 0.9565
Epoch 6/30
557/557 [==============================] - 2s 4ms/step - loss: 0.1006 - accuracy: 0.9649
Epoch 7/30
557/557 [==============================] - 2s 4ms/step - loss: 0.0828 - accuracy: 0.9720
Epoch 8/30
557/557 [==============================] - 2s 4ms/step - loss: 0.0688 - accuracy: 0.9765
Epoch 9/30
557/557 [==============================] - 2s 4ms/step - loss: 0.0567 - accuracy: 0.9805
Epoch 10/30
557/557 [==============================] - 2s 4ms/step - loss: 0.0498 - accuracy: 0.9822

In [ ]:
print('Model evaluation ',Dense_model.evaluate(x_test,y_test))

279/279 [==============================] - 1s 3ms/step - loss: 0.1234 - accuracy: 0.9708
Model evaluation  [0.1233641505241394, 0.9707963466644287]


In [ ]:
print('Model evaluation ',Dense_model.evaluate(x_train,y_train))

557/557 [==============================] - 1s 3ms/step - loss: 0.0145 - accuracy: 0.9952
Model evaluation  [0.01447790302336216, 0.9951701760292053]
